# markdown.obsidian.vault

> Functions for inspecting and modifying Obsidian.md vaults and their folders 

In [ ]:
# TODO: explain what this module is supposed to do.
# TODO: talk about how vault notes are md files.
# TODO: talk about Wikilinks vs Obsidian links

In [ ]:
#| default_exp markdown.obsidian.vault

In [ ]:
#| export
from pathlib import Path
import os
from os import PathLike
from trouver.helper import (
    path_name_no_ext, path_no_ext
)
from typing import Union

In [ ]:
import shutil
import tempfile
from unittest import mock

from fastcore.test import *
from nbdev.showdoc import show_doc

from trouver.helper import path_name_no_ext, _test_directory

## Errors

In [ ]:
#| export
class NoteNotUniqueError(FileNotFoundError):
    """
    A `NoteNotUniqueError` is raised when a `VaultNote` is specified
    by name and not by relative path in the vault, but
    the vault is found to have multiple notes of the name.

    **Attributes**

    - `note_name` - str
        - The name of the note which is not unique in the vault.
    - `notes` - list[str]
        - The paths of the notes whose names are `note_name`.
    """
    def __init__(self, /, *args, **kwargs):
        super().__init__(*args, **kwargs)
            
    @classmethod
    def from_note_names(cls, note_name: str, notes: list[str]):
        """Construct a `NoteNotUniqueError` object from note names"""
        return cls(
            f'The note of the following name is not unique: {note_name}\n'\
            f'The name points to the following files: {notes}')



In [ ]:
show_doc(NoteNotUniqueError.from_note_names)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L34){target="_blank" style="float:right; font-size:smaller"}

### NoteNotUniqueError.from_note_names

>      NoteNotUniqueError.from_note_names (note_name:str, notes:list[str])

Construct a `NoteNotUniqueError` object from note names

In [ ]:
#| export
class NoteDoesNotExistError(FileNotFoundError):
    """
    A `NoteDoesNotExistError` is raised when a `VaultNote` is specified
    by either name and or by relative path in the vault, but
    the vault is found to have no notes of the name.
    """
    def __init__(self, /, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @classmethod
    def from_note_name(cls, note_name: str):
        """Construct a `NoteDoesNotExistError` object from note name"""
        return cls(
            f'The note of the following name does not exist: {note_name}.'
            f' Make sure that the argument passed to `note_name` does not'
            f' erroneously end with `.md`, e.g. pass `this_is_a_note`'
            f' instead of `this_is_a_note.md`.')

## Convert path to Obsidian ID

In [ ]:
#| export
def path_to_obs_id(
        rel_path: PathLike # A path representation the path of an Obsidian note relative to its vault. This does not have to be an existing path.
        ) -> str: # The obsidian url of the hypothetical note within its vault. Note that this does not end with the file extension `.md`.
    """Convert a relative path of an Obsidian note to the Obsidian identifying
    str.
    
    This identification is for a vault-internal Wikilink.

    Note that this function does not have a vault as a parameter.
    """
    path_without_extension = path_no_ext(rel_path)
    return path_without_extension.replace('\\', '/')

Obsidian formats its file paths with '/'; the `path_to_obs_id` function converts a relative path of an Obsidian note to the Obsidian-recognized path.

In [ ]:
test_eq(path_to_obs_id(
    r'some_folder\some_subfolder\some_subsubfolder\some_file.md'),
    'some_folder/some_subfolder/some_subsubfolder/some_file')

Obsidian notes might contain spaces in their paths.

In [ ]:
test_eq(path_to_obs_id(
    Path('some folder\some subfolder\some file.md')),
    'some folder/some subfolder/some file')

## Example vault


We use the following example vault to demonstrate how to use the functions below:

```
.
├── algebra
│   ├── reference_1
│   │   └── ring.md
│   └── ring.md
├── analysis
│   └── exponential_function.md
├── category_theory
│   └── category.md
├── topology
│   └── category.md
└── _index.md
```

Note that it has $6$ files and $5$ folders in total.

In [ ]:
def make_example_vault(temp_dir: PathLike):
    temp_dir = Path(temp_dir)
    os.mkdir(temp_dir / 'algebra')
    os.mkdir(temp_dir / 'algebra' / 'reference_1')
    os.mkdir(temp_dir / 'analysis')
    os.mkdir(temp_dir / 'category_theory')
    os.mkdir(temp_dir / 'topology')
    (temp_dir / 'algebra' / 'ring.md').touch()
    (temp_dir / 'algebra' / 'reference_1' / 'ring.md').touch()
    (temp_dir / 'analysis' / 'exponential_function.md').touch()
    with open((temp_dir / 'analysis' / 'exponential_function.md'), 'w') as writer:
        writer.write(
            r'''The **exponential function** is the function sending a complex number $z$ to **$$e^{z} = \sum_{n=0}^\infty \frac{z^n}{n!}$$**. It converges for all $z \in \mathbb{C}$.'''
        )
    (temp_dir / 'category_theory' / 'category.md').touch()
    (temp_dir / 'topology' / 'category.md').touch()
    (temp_dir / '_index.md').touch()

## Get all notes

In [ ]:
#| export
def all_paths_to_notes_in_vault(
        vault: PathLike, as_dict: bool = False)\
        -> Union[list[str], dict[str, list[str]]]:
    """Return the paths, relative to the Obsidian vault, of notes 
    in the Obsidian vault.
       
    This may not actually return all of the paths to the notes, see
    the parameter `as_dict`.

    **Parameters**

    - `vault` - `PathLike`
        - The path to the Obsidian vault directory
    - `as_dict` - `bool`
        - If `True`, then returns a dict. If `False`, then returns
        a list. Defaults to `False`. If there are multiple notes with the same
        name in the vault, and `as_dict` is set to `True`, then the dictionary
        will contain only one of the (relative) paths to those notes among its
        values. If `as_dict` is set to `False`, then the list will contain
        all the paths to the notes even when notes with non-unique names exist.
        
    **Returns**

    - Union[list[str], dict[str, str]]
        - Each str represents the path relative to `vault`. If `as_dict` is
        True, then returns a dict whose keys are str, which are (unique) names
        of the notes in the vault, and the values are the paths.
    """
    vault = Path(vault)
    paths =  [os.path.relpath(path, vault) for path in vault.glob(f'**/*.md')]
    if as_dict:
        dicty = {path_name_no_ext(path): [] for path in paths}
        for path in paths:
            dicty[path_name_no_ext(path)].append(path)
        return dicty
    else:
        return paths


The `all_paths_to_notes_in_vault` function returns all of the paths to notes in the Obsidian vault; only `.md` files are recognized as notes.

By default, the function returns a list whose items are strings of paths to notes relative to the vault path.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    # assert os.path.exists(_test_directory() / 'test_vault_1')
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)
    # make_example_vault(temp_dir)

    # os.startfile(os.getcwd())

    notes = all_paths_to_notes_in_vault(temp_vault)
    test_eq(len(notes), 6)
    print(notes)
    # test_shuffled(, list)
    

['_index.md', 'algebra\\ring.md', 'algebra\\reference_1\\ring.md', 'analysis\\exponential_function.md', 'category_theory\\category.md', 'topology\\category.md']


Passing `as_dict=True` returns a dictionary whose keys are note names and whose values are lists of paths to notes of the name relative to the vault.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    make_example_vault(temp_dir)
    # os.startfile(os.getcwd())

    notes = all_paths_to_notes_in_vault(temp_dir, as_dict=True)
    test_eq(len(notes), 4)
    test_eq(len(notes['ring']), 2)
    print(notes)


{'_index': ['_index.md'], 'ring': ['algebra\\ring.md', 'algebra\\reference_1\\ring.md'], 'exponential_function': ['analysis\\exponential_function.md'], 'category': ['category_theory\\category.md', 'topology\\category.md']}


## Searching notes by name

In [ ]:
#| export
def all_note_paths_by_name(
        name: str,  # Name of the note(s) to find
        vault: PathLike,  # The path to the Obsidian vault directory
        subdirectory: Union[PathLike, None] = None # The path to a subdirectory in the Obsidian vault, relative to `vault`. If `None`, then denotes the root of the vault.
        ) -> list[Path]: # Each item is a path to a note of the given name, relative to `vault`.
    """Return the relative paths to all notes in the Obsidian vault 
    with the specified name in the specified subdirectory.
    
    This function does not assume that the specified subdirectory in the vault
    has at most one note of the specified name.
    
    """
    vault = vault if vault != None else ''
    vault = Path(vault)
    subdirectory = subdirectory if subdirectory != None else ''
    directory_path = vault / subdirectory
    all_notes_of_name = directory_path.glob(f'**/{name}.md')
    all_notes_of_name = list(all_notes_of_name)
    return [note_path.relative_to(vault) for note_path in all_notes_of_name]

Basic usage:

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    make_example_vault(temp_dir)

    notes = all_note_paths_by_name('ring', temp_dir)
    print('Searched for notes of name `ring`:')
    print(notes, '\n')
    assert len(notes) == 2

    notes = all_note_paths_by_name('exponential_function', temp_dir)
    print('Searched for notes of name `exponential_function`:')
    print(notes, '\n')
    assert len(notes) == 1

    empty_list = all_note_paths_by_name('non-existent-note-name', temp_dir)
    print('Searching for a non-existent note name yields an empty list:')
    print(empty_list)
    assert len(empty_list) == 0

Searched for notes of name `ring`:
[Path('algebra/ring.md'), Path('algebra/reference_1/ring.md')] 

Searched for notes of name `exponential_function`:
[Path('analysis/exponential_function.md')] 

Searching for a non-existent note name yields an empty list:
[]


We can specify a subdirectory inside the vault to restrict the search to.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)

    all_notes_named_curve_in_vault = all_note_paths_by_name('category', vault)
    print('All notes named `category`:\n', all_notes_named_curve_in_vault, '\n')
    assert len(all_notes_named_curve_in_vault) == 2
    for note_path in all_notes_named_curve_in_vault:
        test_eq(path_name_no_ext(note_path), 'category')
    
    print('All notes named `category` in the subdirectory `topology`')
    notes_named_topology_in_subdirectory = all_note_paths_by_name(
        'category', vault, 'topology')
    print(notes_named_topology_in_subdirectory)
    assert len(notes_named_topology_in_subdirectory) == 1
    assert path_name_no_ext(notes_named_topology_in_subdirectory[0]) == 'category'

All notes named `category`:
 [Path('category_theory/category.md'), Path('topology/category.md')] 

All notes named `category` in the subdirectory `topology`
[Path('topology/category.md')]


In [ ]:
#| export
# TODO: include examples of `hints` parameter.
def note_path_by_name(
        name: str, # The path to the Obsidian vault directory.
        vault: PathLike, # The path to a subdirectory in the Obsidian vault. If `None`, then denotes the root of the vault.
        subdirectory: Union[PathLike, None] = None, # The path to a subdirectory in the Obsidian vault. If `None`, then denotes the root of the vault.
        hints: Union[list[PathLike], None] = None # Hints of which directories, relative to `subdirectory` that the note may likely be in. This is for speedup. The directories will be searched in the order listed.
        ) -> Path: # The note of the specified name in the specified subdirectory of the vault.
    """Return the path, relative to a subdirectory in the vault, 
    of the note of the specified name.

    **Raises**

    - NoteNotUniqueError
        - If the note of the specified name is not unique in the subdirectory.
    - NoteDoesNotExistError
        - If the note of the specified name does not exist in the subdirectory.

    **See Also**

    - The constructor of the `VaultNote` class
        - passing an argument to the `name` parameter of this constructor
        method essentially does the same thing as this function, except the
        constructor method uses a cache.
    """
    if not subdirectory:
        subdirectory = ''
    if not hints:
        hints = []
    vault = Path(vault)
    subdirectory = Path(subdirectory)
    # absolute_subdirectory = vault / subdirectory
    hints.append('')  # Search in subdirectory if all else fails
    for hint in hints:
        search_results = all_note_paths_by_name(name, vault, subdirectory / hint)
        if len(search_results) > 1:
            raise NoteNotUniqueError.from_note_names(name, search_results)
        elif len(search_results) == 1:
            return search_results[0]
    raise NoteDoesNotExistError.from_note_name(name)

Assuming that there exists a unique note of a specified name in a vault, we can identify it.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)
    note_path = note_path_by_name('exponential_function', vault)
    print(note_path)
    test_eq(path_name_no_ext(note_path), 'exponential_function')

analysis\exponential_function.md


If there is more than one note in the vault of the specified name, then a `NoteNotUniqueError` is raised.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)
    print(f'The vault has more than one note named `category`:\n',
            all_note_paths_by_name('category', vault))
    with (ExceptionExpected(ex=NoteNotUniqueError, regex='not unique')):
        note_path_by_name('category', vault)

The vault has more than one note named `category`:
 [Path('category_theory/category.md'), Path('topology/category.md')]


Passing an argument to the parameter `subdirectory` restricts the search to the subdirectory. A `NoteNotUniqueError` can be avoided if a subdirectory is specified and if the note of the specified name is unique in the subdirectory.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)
    note = note_path_by_name('category', vault, subdirectory='topology')
    print(note)
    assert 'topology' in str(note) and path_name_no_ext(note) == 'category'


topology\category.md


If there is no note in the vault of the specified name, then a `NoteDoesNotExistError` is raised.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)
    with (ExceptionExpected(ex=NoteDoesNotExistError, regex='does not exist')):
        note_path_by_name('this_note_does_not_exit', vault)

In [ ]:
#| export
def note_name_unique(
        name: str, # Name of the note.
        vault: PathLike # Path to the vault.
        ) -> bool: 
    """Return `True` if a note of the specified name exists and 
    is unique in the Obsidian vault.
    """
    return len(all_note_paths_by_name(name, vault)) == 1

Example use:

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)
    vault_has_unique_note_named_ring = note_name_unique('ring', vault)
    assert not vault_has_unique_note_named_ring

    vault_has_unique_note_named_exponential_function = note_name_unique('exponential_function', vault)
    assert vault_has_unique_note_named_exponential_function

## Getting note name from its path

In [ ]:
#| export
def note_name_from_path(
        note_path: str # The path of the note. The note does not need to exist.
        ) -> str: # The name of the note.
    """Return the name of a note from its path.
    """
    return path_name_no_ext(note_path)

In [ ]:
assert note_name_from_path('algebra/ring.md') == 'ring'

## VaultNote class
Just as how paths in Python can be dealt either via strings of paths or via `pathlib.Path` objects, It is useful to have a class to encapsulate together the name of a note, and its path/Obsidian vault identifier.

In [ ]:
#| export
# TODO: change NoteDoesNotExistError to NoteNotFoundError if not found in Cache.
# TODO: when making NoteNotFoundError, add it to
# markdown.obsidian.personal.reference.delete_reference_folder

# TODO: modify the constructor so that it does not need the note of the specified name to exist
# TODO: look at parts where the cache is accessed/modified. Factor them out into getter/setter methods
# taking in the vault and name as inputs.
# TODO: look at how NoteDoesNotExistError is used and make modifications to account for the modification
# of the constructor.
# TODO: test hidden methods
class VaultNote:
    """Represents a note in an Obsidian vault, without regards to the contents.
    
    The note does not have to exist, except in circumstances stating 
    otherwise.

    TODO go through the methods of this class to see which methods assume that
    the note exists and which do not.

    TODO finish the sentence below.
    A `VaultNote` can be specified by either the `rel_path` or the `name` argument
    in its constructor. If `name` is specified, then the 
    
    TODO implement subdirectory hint
    
    **Attributes**

    - vault - Path
        - The (relative or absolute) path of the Obsidian vault
        that the note is located in.
    - name - str
        - The name of the note in the vault.
    - rel_path - str
        - The note's path relative to the vault. If 
    - cache - dict[str, dict[str, list[str]]], class attribute
        - The keys are string, which are paths to vaults. The
        corresponding values are dict whose keys are string, which are
        names in the vault of the (unique) note of that name, and whose
        values are list of string, which are paths to the note relative to the
        vault. The cache is not automatically updated when notes are
        moved.
    
    **Parameters**

    - vault - PathLike
    - rel_path - PathLike
    - name - str
        - The name of the note in the vault. Defaults to the empty str.
            - If `None`, then the `rel_path` parameter should be used 
            to determine `self.name` instead. 
            - If not `None`, then the note must uniquely exist in the
            vault.
    - `subdirectory` - Union[PathLike, None]
    - `hints` - list[PathLike]
    """
    
    cache = {}
    
    def __init__(
            self,
            vault: PathLike, # The (relative or absolute) path of the Obsidian vault that the note is located in.
            rel_path: PathLike = None, # The note's path relative to the vault. If `None`, then the `name` parameter is used to determine the note instead. Defaults to `None`.
            name: str = None, # The name of the note. If `None`, then the `rel_path` parameter is used to determine the note instead. Defaults to `None` 
            subdirectory: Union[PathLike, None] = '', # The relative path to a subdirectory in the Obsidian vault. If `None`, then denotes the root of the vault. Defaults to the empty str. 
            hints: list[PathLike] = [] # Paths, relative to `subdirectory`, to directories where the note file may be found. This is for speedup. Defaults to the empty list, in which case the vault note is searched in all of `subdirectory`.
            ):
        self.vault = Path(vault)
        assert rel_path or name
        if rel_path is not None:
            self.rel_path = str(rel_path)
            self.name = note_name_from_path(self.rel_path)
        else:
            self.name = name
            self.rel_path = None
            self.identify_rel_path(update_cache=True)

    def obsidian_identifier(self) -> str:
        """Return the Obsidian identifier of the `VaultNote` object.
        
        This is the note's unqiue Obsidian id in the vault. This is like a
        path str with forward slashes `/` (as opposed to backwards `\` slashes)
        and without a file extension (`.md`).
        """
        return path_to_obs_id(self.rel_path)

    def _identify_rel_path(self) -> Union[Path, None]:
        """Returns the Path to the note that this object represents.

        More precisely, if `rel_path` is specified at construction or
        if `self.rel_path` is already identified, then this method
        returns that path. Otherwise, this method looks into the cache to
        check if a note of the `name` specified at construction is in the
        vault and returns the first note in the list of the `name` in the
        cache. If no such note exists, then this method reutrns `None`.
        """
        if self.rel_path is not None:
            return self.rel_path
        cache_search = self.__class__._get_from_cache(self.vault, self.name)
        if cache_search:  # `cache_search` could be `None` or an empty list.
            return cache_search[0]
        return None

    # TODO: give examples of this method
    def identify_rel_path(
            self,
            update_cache=False # If `True`, if the cache is searched, and if a note of the specified name is not found in the cache, then the cache is updated and searched again. Defaults to `False`.
            ) -> None:
        """Sets `self.rel_path` to a path, if not already done so.

        If `self.rel_path` is not already set as a path, then the cache
        is searched to find a note whose name is `self.name` (which is
        necessarily specified).
        """
        rel_path = self._identify_rel_path()
        if rel_path is not None:
            self.rel_path = rel_path
        elif update_cache:
            self.__class__.update_cache(self.vault)
            self.identify_rel_path(update_cache=False)

    @classmethod
    def _get_from_cache(
            cls,
            vault: PathLike,
            name: str) -> Union[list[str], None]:
        """Return the cache's list of notes of the specified name in the
        specified vault.

        If no such list exists in the cache, then return `None`.
        """
        vault = str(vault)
        if vault not in cls.cache:
            return None
        vault_dict = cls.cache[vault]
        if not name in vault_dict:
            return None
        return vault_dict[name]
                        
    def exists(
            self,
            update_cache=False # If `True`, then update the cache and try to identify `self.rel_path` before verifying whether the note exists in the vault.
            ) -> bool:
        """Returns `True` if `self.rel_path` is identified and
        if the note exists in the vault."""
        if self.rel_path is None:
            if update_cache:
                self.identify_rel_path(update_cache=True)
            else:
                return False
        return os.path.exists(self.path())
            
    def path(self,
             relative=False # If `True`, then return the path relative to the vault.
             ) -> Union[Path, None]: # Path to the note if self.rel_path is deterined. `None` otherwise.
        """Returns the path to the note.
        """
        if self.rel_path is None:
            return None
        return Path(self.rel_path) if relative\
            else self.vault / self.rel_path
    
    def directory(self,
                  relative=False # If `True`, then return the path of the directory relative to the vault.
                  ) -> Path: # The path of the directory that the note is in.
        """Return the directory that the note is in.
        """
        rel_dir = Path(os.path.dirname(self.rel_path))
        return rel_dir if relative else self.vault / rel_dir
    
    def create(self):
        """Create the note if it does not exist.
        
        The directory of the file needs to be created separately
        beforehand.

        If the file exists, then a FileExistsError is raised and
        the file modification time is not changed.
        
        **Raises**

        - FileExistsError
            - If the file already exists.
        - FileNotFoundError
            - If the directory of the file does not already exist.
        """
        Path(self.path()).touch(exist_ok=False)
        self.__class__._add_single_entry_to_cache(
            self.vault, self.rel_path)
        
    def move_to(self,
                rel_path: PathLike # The path in which to rename the path to `self` as, relative to `self.vault`.
                ) -> None:
        """Move/rename the note to the specified location in the vault,
        assuming that it exists.
        """
        if not self.exists():
            return
        os.rename(self.path(), Path(self.vault) / rel_path)
        self.__class__._remove_single_entry_from_cache(
            self.vault, Path(self.rel_path))
        self.__class__._add_single_entry_to_cache(
            self.vault, Path(rel_path))
        self.rel_path = str(rel_path)
        self.name = note_name_from_path(self.rel_path)
        
    def move_to_folder(self,
                       rel_dir: PathLike # The path of the directory in which to move `self` to, relative to `self.vault`.
                       ) -> None:
        """Move the note to the specified folder in the vault, assuming that
        if exists.
        """
        self.move_to(Path(rel_dir) / f'{self.name}.md')
        
    def text(self
            ) -> str: # Text contained in the note, assuming that the note exists.
        """Returns the text contained in the note.
        
        **Raises**

        - NoteDoesNotExistError
            - If `self` does not point to an existing note.
        """
        if not self.exists():
            raise NoteDoesNotExistError.from_note_name(self.name)
        with open(self.vault / self.rel_path, 
                  'r', encoding='utf-8') as reader:
            text = reader.read()
            reader.close()
        return text

    @classmethod
    def update_cache(cls,
                     vault: PathLike # The vault.
                     ) -> None:
        """Class method to update cache for `vault` by inspecting all files
        in subdirectories of `vault`."""
        VaultNote.cache[str(vault)] = all_paths_to_notes_in_vault(
            vault, as_dict=True)
        
    @classmethod
    def clear_cache(cls):
        """Class method to clear out the entire cache for all vaults."""
        VaultNote.cache = {}
        
    @classmethod
    def _check_name_exists_and_unique_in_vault_cache(
            cls,
            vault: PathLike, # The vault
            name: str # The name
            ) -> None:
        """
        Raise `NoteNotUniqueError` or `NoteDoesNotExistError` if the note of
        the specified name is not unique or does not exist in the cache.
        
        **Parameters**
        - vault - PathLike
        - name - str

        **Raises**
        - `NoteNotUniqueError`
        - `NoteDoesNotExistError`.
        """
        vault_dict = cls.cache[str(vault)]
        if not name in vault_dict or len(vault_dict[name]) == 0:
            raise NoteDoesNotExistError.from_note_name(name)
        elif len(vault_dict[name]) > 1:
            raise NoteNotUniqueError.from_note_names(
                name, vault_dict[name])

    @classmethod
    def _check_if_cache_needs_to_update(
            cls, vault: PathLike, name: str) -> bool:
        """Returns `True` if the cache needs to update by virtue of not finding
        notes of the specified `name` in the `vault`.
        """
        return not bool(cls._get_from_cache(vault, name))

    @classmethod
    def _add_single_entry_to_cache(
            cls, vault: PathLike, rel_path: PathLike) -> None:
        """Adds a single entry for a note to the cache.
        
        Does nothing if the entry already exists.

        **Parameters**
        - vault - PathLike
        - rel_path - PathLike
            - The path to the note, relative to `vault`.
        """
        vault_str = str(vault)
        if vault_str not in cls.cache:
            cls.cache[vault_str] = {}
        name = note_name_from_path(rel_path)
        if name not in cls.cache[vault_str]:
            cls.cache[vault_str][name] = []
        if rel_path not in cls.cache[vault_str][name]:
            cls.cache[vault_str][name].append(str(rel_path))
    
    @classmethod
    def _remove_single_entry_from_cache(
            cls, vault: PathLike, rel_path: PathLike) -> None:
        """Removes a single entry for a note from the cache.

        Does nothing if the entry is not already there.

        **Parameters**
        - vault - PathLike
        - rel_path - PathLike
            - The path to the note, relative to `vault`.
        """
        vault_str = str(vault)
        if vault_str not in cls.cache:
            return
        name = note_name_from_path(rel_path)
        if name not in cls.cache[vault_str]:
            return
        rel_path = Path(rel_path)
        cls.cache[vault_str][name] = [
            cache_path for cache_path in cls.cache[vault_str][name]
            if Path(cache_path) != rel_path]

    @classmethod
    def unique_name(
            cls,
            name: str, # The base name for the note.
            vault: PathLike # The vault
            ) -> str: # A str obtained by appending `_{some number}` to the end of `name`.
        """A class method to return a name for a note that is unique in
        the vault based on a specified name.
        """
        if not str(vault) in VaultNote.cache:
            cls.update_cache(vault)
        if not name in VaultNote.cache[str(vault)]:
            return name
        i = 1
        while f'{name}_{i}' in VaultNote.cache[str(vault)]:
            i += 1
        return f'{name}_{i}'

## Functions/Methods of the `VaultNote` class

In [ ]:
show_doc(VaultNote.obsidian_identifier)

---

### VaultNote.obsidian_identifier

>      VaultNote.obsidian_identifier ()

Return the Obsidian identifier of the `VaultNote` object.

This is the note's unqiue Obsidian id in the vault. This is like a
path str with forward slashes `/` (as opposed to backwards `\` slashes)
and without a file extension (`.md`).

In [ ]:
show_doc(VaultNote.identify_rel_path)

---

### VaultNote.identify_rel_path

>      VaultNote.identify_rel_path (update_cache=False)

Sets `self.rel_path` to a path, if not already done so.

If `self.rel_path` is not already set as a path, then the cache
is searched to find a note whose name is `self.name` (which is
necessarily specified).

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| update_cache | bool | False | If `True`, if the cache is searched, and if a note of the specified name is not found in the cache, then the cache is updated and searched again. Defaults to `False`. |
| **Returns** | **None** |  |  |

In [ ]:
show_doc(VaultNote.exists)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L278){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.exists

>      VaultNote.exists (update_cache=False)

Returns `True` if `self.rel_path` is identified and
if the note exists in the vault.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| update_cache | bool | False | If `True`, then update the cache and try to identify `self.rel_path` before verifying whether the note exists in the vault. |
| **Returns** | **bool** |  |  |

In [ ]:
show_doc(VaultNote.path)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L282){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.path

>      VaultNote.path (relative=False)

Returns the path to the note.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| relative | bool | False | If `True`, then return the path relative to the vault. |
| **Returns** | **Optional** |  | **Path to the note if self.rel_path is deterined. `None` otherwise.** |

In [ ]:
show_doc(VaultNote.directory)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L290){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.directory

>      VaultNote.directory (relative=False)

Return the directory that the note is in.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| relative | bool | False | If `True`, then return the path of the directory relative to the vault. |
| **Returns** | **Path** |  | **The path of the directory that the note is in.** |

In [ ]:
show_doc(VaultNote.create)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L298){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.create

>      VaultNote.create ()

Create the note if it does not exist.

The directory of the file needs to be created separately
beforehand.

If the file exists, then a FileExistsError is raised and
the file modification time is not changed.

**Raises**

- FileExistsError
    - If the file already exists.
- FileNotFoundError
    - If the directory of the file does not already exist.

In [ ]:
show_doc(VaultNote.move_to)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L319){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.move_to

>      VaultNote.move_to (rel_path:os.PathLike)

Move/rename the note to the specified location in the vault,
assuming that it exists.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| rel_path | PathLike | The path in which to rename the path to `self` as, relative to `self.vault`. |
| **Returns** | **None** |  |

In [ ]:
show_doc(VaultNote.move_to_folder)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L336){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.move_to_folder

>      VaultNote.move_to_folder (rel_dir:os.PathLike)

Move the note to the specified folder in the vault, assuming that
if exists.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| rel_dir | PathLike | The path of the directory in which to move `self` to, relative to `self.vault`. |
| **Returns** | **None** |  |

In [ ]:
show_doc(VaultNote.text)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L343){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.text

>      VaultNote.text ()

Returns the text contained in the note.

**Raises**

- NoteDoesNotExistError
    - If `self` does not point to an existing note.

In [ ]:
show_doc(VaultNote.update_cache)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L361){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.update_cache

>      VaultNote.update_cache (vault:os.PathLike)

Class method to update cache for `vault` by inspecting all files
in subdirectories of `vault`.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| vault | PathLike | The vault. |
| **Returns** | **None** |  |

In [ ]:
show_doc(VaultNote.clear_cache)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L370){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.clear_cache

>      VaultNote.clear_cache ()

Class method to clear out the entire cache for all vaults.

In [ ]:
show_doc(VaultNote.unique_name)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L459){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.unique_name

>      VaultNote.unique_name (name:str, vault:os.PathLike)

A class method to return a name for a note that is unique in
the vault based on a specified name.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| name | str | The base name for the note. |
| vault | PathLike | The vault |
| **Returns** | **str** | **A str obtained by appending `_{some number}` to the end of `name`.** |

## Constructing VaultNote instances

#### By name

The most convenient way to construct an existing `VaultNote` is to specify the vault in which it exists and the note's name. 

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)

    vault_note = VaultNote(vault, name='exponential_function')
    assert vault_note.exists()
    print(vault_note.name)
    print(f'`vault_note` is located, relative to `vault`, at {vault_note.rel_path}.')

exponential_function
`vault_note` is located, relative to `vault`, at analysis\exponential_function.md.


In [ ]:
# TODO: Delete the below example
# If an argument is passed to the `name` parameter and if the note's name is not unique in the vault, then a `NoteNotUniqueError` is raised. 

# If an argument is passed to the `name` parameter and if the note's name does not exist in the vault, then a `NoteDoesNotExistError` is raised.
# with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
#     make_example_vault(vault)

#     with ExceptionExpected(ex=NoteNotUniqueError):
#         vault_note = VaultNote(vault, name='ring')
#     with ExceptionExpected(ex=NoteDoesNotExistError):
#         vault_note = VaultNote(vault, name='does_not_exist')

#### By relative path

Alternatively, a `VaultNote` object can be created by passing an argument to the `rel_path` parameter. In this case, the note of the specified path does not need to exist.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)

    vault_note = VaultNote(vault, rel_path='non_existent_folder/non_existent_note.md')
    assert not vault_note.exists()
    # Note that there is not a unique note of name `ring`.
    vault_note = VaultNote(vault, rel_path='algebra/ring.md')  
    assert vault_note.exists()

The `rel_path` parameter takes precedence over the `name` parameter.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)
    vault_note = VaultNote(vault, rel_path='non_existent_folder/non_existent_note.md', name='ring')
    assert vault_note.name == 'non_existent_note'

#### Cache of the `VaultNote` class

The `VaultNote` class keeps a cache of the notes (files with extension `.md`) in the vault. In pracctice, this cache is updated when a note of the specified name is not found when constructing a `VaultNote` instance via the `name` parameter.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)
    VaultNote.update_cache(vault)
    
    assert VaultNote._check_name_exists_and_unique_in_vault_cache(vault, 'exponential_function') is None
    with ExceptionExpected(ex=NoteDoesNotExistError):
        VaultNote._check_name_exists_and_unique_in_vault_cache(vault, 'this_note_does_not_exist')
        # TODO: test this works correctly when a note does not exist by being removed.
    with ExceptionExpected(ex=NoteNotUniqueError):
        VaultNote._check_name_exists_and_unique_in_vault_cache(vault, 'ring')

In [ ]:
VaultNote.clear_cache()
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)
    assert VaultNote._check_if_cache_needs_to_update(vault, name='ring')
    VaultNote.update_cache(vault)
    assert VaultNote._check_if_cache_needs_to_update(vault, name='does_not_exist')

## Getting information about the note

Here are some convenient ways to get information about the `VaultNote`:

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)
    vault_note = VaultNote(vault, name='exponential_function')
    print(f'Obsidian vault identifier:\t{vault_note.obsidian_identifier()}')
    print(f'relative path:\t{vault_note.rel_path}')
    print(f'Part of the absolute path of the note:\t{str(vault_note.path(relative=False))[:7]}')
    print(f'note name:\t{vault_note.name}')
    print(f'directory that the note is in relative to the vault:\t{vault_note.directory(relative=True)}')

Obsidian vault identifier:	analysis/exponential_function
relative path:	analysis\exponential_function.md
Part of the absolute path of the note:	c:\User
note name:	exponential_function
directory that the note is in relative to the vault:	analysis


## Reading the contents of the note

The `VaultNote.text()` function reads the contents of the file that the `VaultNote` object represents, assuming that this file exists. If the file does not exist, then a `NoteDoesNotExistError` is raised.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)
    vault_note = VaultNote(vault, name='exponential_function')
    print(vault_note.text())
    assert len(vault_note.text()) > 10

    vault_note = VaultNote(vault, rel_path='does_not_exist.md')
    with ExceptionExpected(ex=NoteDoesNotExistError):
        vault_note.text()

The **exponential function** is the function sending a complex number $z$ to **$$e^{z} = \sum_{n=0}^\infty \frac{z^n}{n!}$$**. It converges for all $z \in \mathbb{C}$.


## Creating/moving the note

If a `VaultNote` object represents a non-existent file, then the file can be created with empty content. The cache is updated with this single new entry, but the rest of the cache remains the same.

If the file exists, then a `FileExistsError` is raised.

If the specified directory for the file does not exist, then a `FileNotFoundError` is raised.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)
    vault_note = VaultNote(vault, rel_path='new_file.md')
    vault_note.create()
    assert vault_note.exists()
    assert vault_note.rel_path in VaultNote.cache[str(vault)]['new_file']

    with ExceptionExpected(ex=FileExistsError):
        vault_note.create() 

    vault_note = VaultNote(vault, rel_path='none_existent_folder/new_file.md')
    with ExceptionExpected(ex=FileNotFoundError):
        vault_note.create()

If a `VaultNote` object represents an existing file, then the file can be renamed or moved.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)

    VaultNote.clear_cache()
    vault_note = VaultNote(vault, name='exponential_function')
    vault_note.move_to_folder('')
    assert vault_note.rel_path == 'exponential_function.md'
    assert 'exponential_function.md' in VaultNote.cache[str(vault)]['exponential_function']

### Getting a unique note name

It is troublesome to create a note with a non-unique name. The `unique_name` method of the VaultNote class takes a tentative name for a note to be created and adds a number to the name so that the note name will be unique in the vault. 

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    make_example_vault(vault)
    # There is a note named category in the test vault.
    sample_name = VaultNote.unique_name('category', vault)  
    assert not VaultNote(vault, name=sample_name).exists()

    sample_name = VaultNote.unique_name('non_existent_note_name', vault)
    assert not VaultNote(vault, name=sample_name).exists()
    assert sample_name == 'non_existent_note_name'

In [ ]:
#| hide
# This import 
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as vault:
    vault = Path(vault)
    make_example_vault(vault)
    # Test _add_single_entry_to_cache
    VaultNote.clear_cache()
    VaultNote._add_single_entry_to_cache(vault, rel_path='directory/file.md')
    assert len(VaultNote.cache) == 1
    assert len(VaultNote.cache[str(vault)]) == 1
    assert len(VaultNote.cache[str(vault)]['file']) == 1

    VaultNote._remove_single_entry_from_cache(vault, rel_path=Path('directory/file.md'))
    assert len(VaultNote.cache) == 1
    assert len(VaultNote.cache[str(vault)]) == 1
    assert len(VaultNote.cache[str(vault)]['file']) == 0
    # Test update_cache with mock
    with mock.patch('__main__.all_paths_to_notes_in_vault') as mock_method:
        VaultNote.update_cache(vault)
        mock_method.assert_called()

    # with (mock.patch.object(VaultNote, '_add_single_entry_to_cache') as mock_add_cache,
    #       mock.patch.object(VaultNote, '_remove_single_entry_from_cache') as mock_remove_cache,
    #       ):

    # See https://stackoverflow.com/questions/29152170/what-is-the-difference-between-mock-patch-object-and-mock-patch
    with (mock.patch('__main__.VaultNote._add_single_entry_to_cache') as mock_add_cache,
          mock.patch('__main__.VaultNote._remove_single_entry_from_cache') as mock_remove_cache,
          ):
        vault_note = VaultNote(vault, name='exponential_function')
        vault_note.move_to_folder('')
        # mock_add_cache.assert_called_with()
        mock_remove_cache.assert_called_with(vault, Path('analysis') / Path('exponential_function.md'))
        mock_add_cache.assert_called_with(vault, Path('exponential_function.md'))